# Build_GNN_2: message passing

In [2]:
import numpy as np
from MPNN_featurize import featurize
from build_GNN_1 import ProteinFeatures, gather_edges, PositionalEncodings

batch = [
    {
        'seq_chain_A': 'MKLVFLVLLVFVQGF',
        'coords_chain_A': {'N_chain_A': np.random.rand(15, 3), 'CA_chain_A': np.random.rand(15, 3), 'C_chain_A': np.random.rand(15, 3), 'O_chain_A': np.random.rand(15, 3)},
        'seq_chain_B': 'MSVKVEEVG',
        'coords_chain_B': {'N_chain_B': np.random.rand(9, 3), 'CA_chain_B': np.random.rand(9, 3), 'C_chain_B': np.random.rand(9, 3), 'O_chain_B': np.random.rand(9, 3)},
        'seq_chain_C': 'ATCGATCGATCGATCG',
        'coords_chain_C': {'N_chain_C': np.random.rand(16, 3), 'CA_chain_C': np.random.rand(16, 3), 'C_chain_C': np.random.rand(16, 3), 'O_chain_C': np.random.rand(16, 3)},
        'masked_list': ['A', 'B'],
        'visible_list': ['C'],
        'num_of_chains': 3,
        'seq': 'MKLVFLVLLVFVQGF'+ 'MSVKVEEVG' + 'ATCGATCGATCGATCG'
    },
      {
        'seq_chain_X': 'ACDEFGHIKLMNPQRSTVWY',
        'coords_chain_X': {'N_chain_X': np.random.rand(20, 3), 'CA_chain_X': np.random.rand(20, 3), 'C_chain_X': np.random.rand(20, 3), 'O_chain_X': np.random.rand(20, 3)},
        'seq_chain_Y': 'ACCDEFGHILKLM',
        'coords_chain_Y': {'N_chain_Y': np.random.rand(13, 3), 'CA_chain_Y': np.random.rand(13, 3), 'C_chain_Y': np.random.rand(13, 3), 'O_chain_Y': np.random.rand(13, 3)},
        'seq_chain_Z': 'LKLMNRPQRST',
        'coords_chain_Z': {'N_chain_Z': np.random.rand(11, 3), 'CA_chain_Z': np.random.rand(11, 3), 'C_chain_Z': np.random.rand(11, 3), 'O_chain_Z': np.random.rand(11, 3)},
        'masked_list': ['X', 'Y'],
        'visible_list': ['Z'],
        'num_of_chains': 3,
        'seq': 'ACDEFGHIKLMNPQRSTVWY'+'ACCDEFGHILKLM'+'LKLMNRPQRST'

    }
]
device='cuda'
X, S, mask, lengths, chain_M, residue_idx, mask_self, chain_encoding_all = featurize(batch, device)
demo = ProteinFeatures(
    edge_features=16, node_features=16, num_positional_embeddings=16,
    num_rbf=16, top_k=30, augment_eps=0., num_chain_embeddings=16
).to(device)
E, E_idx=demo.forward(X,chain_M, residue_idx, chain_encoding_all)

In [ ]:
# initialize h_V with zero match h_E dim
import torch
h_V = torch.zeros((E.shape[0], E.shape[1], E.shape[-1]), device=E.device)
# MLP of h_E, we omit it here
edge_features=E.shape[-1],
hidden_dim=16
W_e = torch.nn.Linear(edge_features, hidden_dim, bias=True).to(device)
h_E = W_e(E)
print(h_E.shape)

# mask  [B, L, L]
def gather_nodes(nodes, neighbor_idx):
    # Features [B,L,C] at Neighbor indices [B,L,K] => [B,N,K,C]
    # Flatten and expand indices per batch [B,L,K] => [B,LK] => [B,NK,C]
    neighbors_flat = neighbor_idx.view((neighbor_idx.shape[0], -1)) # [B, LK]
    neighbors_flat = neighbors_flat.unsqueeze(-1).expand(-1, -1, nodes.size(2)) #[B,Lk,1]->[b,lk,l]
    # Gather and re-pack
    # nodes [B,L,L,1]
    neighbor_features = torch.gather(nodes, 1, neighbors_flat)
    neighbor_features = neighbor_features.view(list(neighbor_idx.shape)[:3] + [-1])
    return neighbor_features

# Encoder is unmasked self-attention
mask_attend = gather_nodes(mask.unsqueeze(-1),  E_idx).squeeze(-1) # [B,L,K]
mask_attend = mask.unsqueeze(-1) * mask_attend
